# Production Service

This notebook will deploy a model (built else where) and deploy it to a REST end point
which will perform model inference.

Steps

1. Build a model in another notebook and note the model name
2. Deploy model to a Production EndPoint (a REST service)
3. (optional) Write a program that communicates with Production EndPoint via REST protocol
4. Shutdown Production EndPoint

In [24]:
from sbrain.serving.model_service_endpoint import *
from sbrain.learning.experiment import *
import time
import os
import logging

## Please change model name (model must exist) and end point name

In [25]:
model_name = "MNIST_Model_15398983879550533"
end_point_name="ep_2"

In [26]:
model = Model.lookup(model_name)
print(str(model.model_metrics))

{"loss": 0.4558957517147064, "global_step": 201, "accuracy": 0.8539999723434448}


## Bring up REST Inference Service Endpoint

In [27]:
mep = ModelEndpoint.create(model=model, endpoint_name=end_point_name,
                            description=end_point_name, number_of_service_replicas=1, gpu_required=True)

mep.search(endpoint_name=end_point_name)
print()
print("EndPoint is up - " + mep.service_url + "model/inference")



Endpoint |  Model   | Author | Created  | Status  | Descript | Service | Service
  Name   |   Name   |        |   Date   |         |   ion    |   URL   | Replica
         |          |        |          |         |          |         |    s   
=========+==========+========+==========+=========+==========+=========+========
ep_2     | MNIST_Mo | admin  | 2018-10- | Started | ep_2     | http:// | 1      
         | del_1539 |        | 19 UTC   |         |          | www.sds |        
         | 89838795 |        |          |         |          | ra2.com |        
         | 50533    |        |          |         |          | /ricky1 |        
         |          |        |          |         |          | 0180208 |        
         |          |        |          |         |          | -ep_2/  |        
EndPoint is up - http://www.sdsra2.com/ricky10180208-ep_2/model/inference


## An example program template to call this REST service from outside the cluster

The below code can be used as an example for invoking a serving interface. Please mind the below things.

1. This code is meant to serve as a client to test models built with Keras Mnist notebook at sample-notebooks/experiment-management/3_KerasTraining.ipynb.
2. This code has to be run outside of the cluster. Running in this notebook will not work. Also this downloads mnist data from internet, so make sure you have an internet connection.
3. Please make sure, that you change the paths marked under "PATHS FOR YOU TO CHANGE" before running.


In [ ]:
# import requests, json
# import base64
# import cv2
# import numpy as np
# from tensorflow.examples.tutorials.mnist import input_data

# ## - PATHS FOR YOU TO CHANGE
# local_mnist_directory = "/tmp"
# encoded_img_path = "/tmp/encoded.jpg"
# url_to_service = "http://sbrain.sdspass.io/sbraindec07-ep_2/model/inference"



# def encode_img(img_path):
#     with open(img_path, "rb") as imageFile:
#         str = base64.b64encode(imageFile.read())
#         print("BASE64 Byte format ... ")
#         print(str)
#         print("BASE64 String format ... ")
#         print(str.decode("utf-8"))
#         return str.decode("utf-8")

# def send_request_to_model_inference_service(message_data):

#     headers = {
#         'Content-Type': 'application/json',
#     }
#     features = []
#     for image in message_data:
#         features.append(image)

#     payload = {
#         'features': features

#     }

#     response = requests.post(url_to_service, headers=headers, data=json.dumps(payload))
#     response.raise_for_status()
#     return response.json()


# def test():
#     mnist = input_data.read_data_sets(local_mnist_directory, one_hot=True)
#     img = mnist.train.images[0, :]
#     cv2.imwrite(encoded_img_path, img.reshape((1, 784)), [cv2.IMWRITE_JPEG_QUALITY, 90])
#     dt = encode_img(encoded_img_path)

#     res = send_request_to_model_inference_service([dt])
#     action_for_first_image = res["actions"][0]
#     prediction = action_for_first_image["dense_1"] ## dense_1 is dependent on the model function that created this model.
#     logits = np.array(prediction)
#     class_response = np.argmax(logits)

# test()

## Bring down REST Inference Service Endpoint

In [28]:
mep2 = ModelEndpoint.lookup(endpoint_name=end_point_name)
mep2.shutdown()
mep2.search(endpoint_name=end_point_name)
time.sleep(15) # allow kubernetes to bring down the container



Endpoint |  Model   | Author | Created  | Status  | Descript | Service | Service
  Name   |   Name   |        |   Date   |         |   ion    |   URL   | Replica
         |          |        |          |         |          |         |    s   
=========+==========+========+==========+=========+==========+=========+========
ep_2     | MNIST_Mo | admin  | 2018-10- | Stopped | ep_2     | http:// | 1      
         | del_1539 |        | 19 UTC   |         |          | www.sds |        
         | 89838795 |        |          |         |          | ra2.com |        
         | 50533    |        |          |         |          | /ricky1 |        
         |          |        |          |         |          | 0180208 |        
         |          |        |          |         |          | -ep_2/  |        
